# Transformer 실습

이번 실습에서는 감정 분석 task에 RNN 대신 Transformer를 구현하여 적용해 볼 것입니다.
Library import나 dataloader 생성은 RNN 실습 때와 똑같기 때문에 설명은 넘어가도록 하겠습니다.

In [1]:
%pip install datasets sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.9 MB/s eta 0:00:0031m19.2 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 15.4 MB/s eta 0:00:001m15.0 MB/s eta 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Users/gimga-eun/.pyenv/versions/3.12.9/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch  # PyTorch 딥러닝 프레임워크
from datasets import load_dataset  # Hugging Face의 데이터셋 로드 도구
from torch.utils.data import DataLoader  # 데이터 로딩을 위한 PyTorch 도구
from transformers import BertTokenizerFast  # BERT 토크나이저
from tokenizers import (  # 토크나이저 관련 도구들
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

# IMDB 영화 리뷰 데이터셋 로드
# train_ds: 학습용 데이터셋
train_ds = load_dataset("stanfordnlp/imdb", split="train[:5%]")
# test_ds: 테스트용 데이터셋
test_ds = load_dataset("stanfordnlp/imdb", split="test[:5%]")

# BERT 토크나이저 로드 (소문자 처리된 버전)
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

# 배치 데이터 처리를 위한 함수
def collate_fn(batch):
    max_len = 400  # 최대 텍스트 길이 설정
    texts, labels = [], []  # 텍스트와 레이블을 저장할 리스트 초기화
    
    # 배치의 각 항목에서 텍스트와 레이블 추출
    for row in batch:
        labels.append(row['label'])  # 레이블 추가 (긍정/부정)
        texts.append(row['text'])    # 텍스트 추가 (영화 리뷰)

    # 텍스트를 토큰화하고 텐서로 변환
    texts = torch.LongTensor(tokenizer(
        texts,
        padding=True,      # 패딩 적용 (길이 통일)
        truncation=True,   # 최대 길이 초과 시 자르기
        max_length=max_len # 최대 길이 설정
    ).input_ids)
    
    # 레이블을 텐서로 변환
    labels = torch.LongTensor(labels)

    return texts, labels

# 학습용 데이터 로더 설정
train_loader = DataLoader(
    train_ds,           # 학습 데이터셋
    batch_size=64,      # 배치 크기
    shuffle=True,       # 데이터 섞기 활성화: 과적합 방지
    collate_fn=collate_fn  # 배치 처리 함수
)

# 테스트용 데이터 로더 설정
test_loader = DataLoader(
    test_ds,            # 테스트 데이터셋
    batch_size=64,      # 배치 크기
    shuffle=False,      # 데이터 섞기 비활성화
    collate_fn=collate_fn  # 배치 처리 함수
)

# DataLoader가 필요한 이유
# 배치 처리 : 전체 데이터를 한 번에 처리하면 메모리 부족 발생, 작은 배치로 나누어 효율적으로 처리
# 데이터 형식 변환 : 원본 데이터(텍스트, 이미지 등)를 모델이 이해할 수 있는 형태로 변환, PyTorch 텐서 형태로 변환 필요

# 전처리
# 데이터를 정규화 하고 일관된 형식으로 만든다.
# 여기서는 최대 길이 통일이나, padding을 통해 길이를 맞추고 있다.


Using cache found in /Users/gimga-eun/.cache/torch/hub/huggingface_pytorch-transformers_main


## Self-attention

이번에는 self-attention을 구현해보겠습니다.
Self-attention은 shape이 (B, S, D)인 embedding이 들어왔을 때 attention을 적용하여 새로운 representation을 만들어내는 module입니다.
여기서 B는 batch size, S는 sequence length, D는 embedding 차원입니다.
구현은 다음과 같습니다.

In [4]:
from torch import nn
from math import sqrt

# self attention
# 입력 시퀀스의 각 위치가 다른 모든 위치와 어떻게 관련되어 있는지 계산
# 문맥을 고려한 표현을 학습
class SelfAttention(nn.Module):  # PyTorch의 nn.Module을 상속받는 SelfAttention 클래스 정의
  def __init__(self, input_dim, d_model):  # 초기화 함수: input_dim(입력 차원)과 d_model(모델 차원) 매개변수
      super().__init__()  # 부모 클래스(nn.Module) 초기화
      
      # 클래스 변수 설정
      self.input_dim = input_dim  # 입력 차원 저장
      self.d_model = d_model      # 모델 차원 저장
      
      # Query, Key, Value를 위한 선형 변환 레이어들
      self.wq = nn.Linear(input_dim, d_model)  # Query 변환
      self.wk = nn.Linear(input_dim, d_model)  # Key 변환
      self.wv = nn.Linear(input_dim, d_model)  # Value 변환
      self.dense = nn.Linear(d_model, d_model) # 출력 변환
      
      # 소프트맥스
      # 모든 값을 0~1 사이로 변환, 모든 값의 합이 1이 되도록 함
      # 각 단어가 다른 단어들과 얼마나 관련있는지를 확률로 표현
      # 높은 점수는 높은 관련성을, 낮은 점수는 낮은 관련성을 의미
      # 중요한 부분에는 높은 가중치를, 덜 중요한 부분에는 낮은 가중치를 부여한다.
      self.softmax = nn.Softmax(dim=-1)  # 어텐션 스코어를 확률로 변환하기 위한 softmax
      
  def forward(self, x, mask):  # 순전파 함수: 입력 x와 mask를 받음
      # Query, Key, Value 생성
      q, k, v = self.wq(x), self.wk(x), self.wv(x)
      
      # 어텐션 스코어 계산: Q와 K의 내적
      score = torch.matmul(q, k.transpose(-1, -2))  # (배치, 시퀀스길이, 시퀀스길이) 크기의 행렬 생성
      
      # 스케일링: d_model의 제곱근으로 나누기
      # 내적 값이 너무 커지는 것을 방지합니다
      # 값이 커지면 softmax 함수에서 기울기가 매우 작아지는 문제(vanishing gradient)가 발생할 수 있습니다
      # d_model의 제곱근으로 나누면 값들이 적절한 범위로 조정됩니다
      score = score / sqrt(self.d_model)
      
      # 마스킹 적용 (필요한 경우)
      # 패딩된 부분이나 미래의 단어를 가리기 위해 사용됩니다
      # -1e9(매우 작은 값)를 더하면 softmax 후에 해당 위치의 가중치가 0에 가까워집니다
      # 이를 통해 특정 위치의 정보를 무시할 수 있습니다
      if mask is not None:
          score = score + (mask * -1e9)  # 마스크 위치에 매우 작은 값을 더해 softmax 후 0에 가까워지게 함

      # 스케일링과 마스킹을 통해 학습을 안정화하고 필요한 정보만 선택적으로 활용할 수 있게 됩니다
          
      # softmax 적용하여 어텐션 가중치 생성
      score = self.softmax(score)
      
      # 가중치와 Value를 곱하여 최종 출력 계산
      result = torch.matmul(score, v)
      
      # 최종 선형 변환 적용
      result = self.dense(result)
      
      return result  # 최종 결과 반환

대부분은 Transformer 챕터에서 배운 수식들을 그대로 구현한 것에 불과합니다.
차이점은 `mask`의 존재여부입니다.
이전 챕터에서 우리는 가변적인 text data들에 padding token을 붙여 하나의 matrix로 만든 방법을 배웠습니다.
실제 attention 계산에서는 이를 무시해주기 위해 mask를 만들어 제공해주게 됩니다.
여기서 mask의 shape은 (B, S, 1)로, 만약 `mask[i, j] = True`이면 그 변수는 padding token에 해당한다는 뜻입니다.
이러한 값들을 무시해주는 방법은 shape이 (B, S, S)인 `score`가 있을 때(수업에서 배운 $A$와 동일) `score[i, j]`에 아주 작은 값을 더해주면 됩니다. 아주 작은 값은 예를 들어 `-1000..00 = -1e9` 같은 것이 있습니다.
이렇게 작은 값을 더해주고 나면 softmax를 거쳤을 때 0에 가까워지기 때문에 weighted sum 과정에서 padding token에 해당하는 `v` 값들을 무시할 수 있게 됩니다.

다음은 self-attention과 feed-forward layer를 구현한 모습입니다.

In [5]:
class TransformerLayer(nn.Module):
  """트랜스포머의 기본 레이어를 구현하는 클래스"""
  
  def __init__(self, input_dim, d_model, dff):
      """
      input_dim: 입력 차원
      d_model: 모델의 내부 차원
      dff: Feed-Forward Network의 은닉층 차원
      """
      super().__init__()  # nn.Module 초기화

      # 클래스 변수 저장
      self.input_dim = input_dim  # 입력 데이터의 차원
      self.d_model = d_model      # 모델 내부에서 사용할 차원
      self.dff = dff              # Feed-Forward Network의 은닉층 크기

      # Self-Attention 레이어
      self.sa = SelfAttention(input_dim, d_model)
      
      # Feed-Forward Network (FFN) 레이어
      self.ffn = nn.Sequential(
          nn.Linear(d_model, dff),     # 첫 번째 선형 변환: d_model -> dff
          nn.ReLU(),                   # 활성화 함수
          nn.Linear(dff, d_model)      # 두 번째 선형 변환: dff -> d_model
      )

  def forward(self, x, mask):
      """
      순전파 함수
      x: 입력 텐서
      mask: 어텐션에서 사용할 마스크
      """
      x = self.sa(x, mask)    # Self-Attention 적용
      x = self.ffn(x)         # Feed-Forward Network 적용

      return x                # 최종 출력 반환

보시다시피 self-attention의 구현이 어렵지, Transformer layer 하나 구현하는 것은 수업 때 다룬 그림과 크게 구분되지 않는다는 점을 알 수 있습니다.

## Positional encoding

이번에는 positional encoding을 구현합니다. Positional encoding의 식은 다음과 같습니다:
$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

이를 Numpy로 구현하여 PyTorch tensor로 변환한 모습은 다음과 같습니다:

In [6]:
import numpy as np

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

# positional encoding의 목적
# 트랜스포머 모델에 순서 정보 제공
# 각 위치마다 고유한 패턴 생성
# sin과 cos 함수를 사용하여 상대적 위치 정보 표현
# positional encoding을 통해 transformer가 입력 시퀀스의 순서 정보를 이해할 수 있다.
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


Positional encoding은 `angle_rads`를 구현하는 과정에서 모두 구현이 되었습니다. 여기서 `angle_rads`의 shape은 (S, D)입니다.
우리는 일반적으로 batch로 주어지는 shape이 (B, S, D)인 tensor를 다루기 때문에 마지막에 None을 활용하여 shape을 (1, S, D)로 바꿔주게됩니다.

위에서 구현한 `TransformerLayer`와 positional encoding을 모두 합친 모습은 다음과 같습니다:

In [13]:
class TextClassifier(nn.Module):
  """텍스트 분류를 위한 트랜스포머 기반 모델"""
    
  def __init__(self, vocab_size, d_model, n_layers, dff):
    """
    vocab_size: 어휘 사전의 크기
    d_model: 모델의 임베딩 차원
    n_layers: 트랜스포머 레이어의 수
    dff: Feed-Forward Network의 은닉층 차원
    """
    super().__init__()

    # 모델 파라미터 저장
    self.vocab_size = vocab_size  # 어휘 사전 크기
    self.d_model = d_model        # 모델 차원
    self.n_layers = n_layers      # 레이어 수
    self.dff = dff                # FFN 차원

    # 단어 임베딩
    # 단어를 고정된 크기의 벡터로 변환하는 과정
    # 텍스트를 숫자로 표현하여 모델이 처리할 수 있게 함
    # 비슷한 의미의 단어는 비슷한 백터를 가지므로 의미적 유사성을 표현할 수 있다.
    self.embedding = nn.Embedding(vocab_size, d_model)
    
    # 위치 인코딩 (학습되지 않는 고정된 파라미터)
    self.pos_encoding = nn.parameter.Parameter(
        positional_encoding(max_len, d_model), 
        requires_grad=False
    )
    
    # 트랜스포머 레이어들
    self.layers = nn.ModuleList([
        TransformerLayer(d_model, d_model, dff) 
        for _ in range(n_layers)
    ])
    
    # 최종 분류를 위한 선형 레이어 (이진 분류)
    self.classification = nn.Linear(d_model, 1)

  def forward(self, x):
    """
    순전파 함수
    x: 입력 텍스트 (토큰 ID)
    """
    # 패딩 마스크 생성 (패딩 토큰 위치 표시)
    mask = (x == tokenizer.pad_token_id)
    mask = mask[:, None, :]  # 어텐션을 위한 차원 추가
    
    seq_len = x.shape[1]  # 시퀀스 길이

    # 임베딩 적용
    x = self.embedding(x)
    # 임베딩 스케일링
    x = x * sqrt(self.d_model)
    # 위치 인코딩 추가
    x = x + self.pos_encoding[:, :seq_len]

    # 트랜스포머 레이어들 순차적으로 적용
    for layer in self.layers:
        x = layer(x, mask)

    # 첫 번째 토큰([CLS]) 선택
    x = x[:, 0]
    # 최종 분류 (이진 분류)
    x = self.classification(x)

    return x


# 모델 인스턴스 생성
model = TextClassifier(
    vocab_size=len(tokenizer),  # 토크나이저의 어휘 사전 크기
    d_model=32,                 # 모델 차원
    n_layers=2,                 # 트랜스포머 레이어 수
    dff=32                      # FFN 차원
)

기존과 다른 점들은 다음과 같습니다:
1. `nn.ModuleList`를 사용하여 여러 layer의 구현을 쉽게 하였습니다.
2. Embedding, positional encoding, transformer layer를 거치고 난 후 마지막 label을 예측하기 위해 사용한 값은 `x[:, 0]`입니다. 기존의 RNN에서는 padding token을 제외한 마지막 token에 해당하는 representation을 사용한 것과 다릅니다. 이렇게 사용할 수 있는 이유는 attention 과정을 보시면 첫 번째 token에 대한 representation은 이후의 모든 token의 영향을 받습니다. 즉, 첫 번째 token 또한 전체 문장을 대변하는 의미를 가지고 있다고 할 수 있습니다. 그래서 일반적으로 Transformer를 text 분류에 사용할 때는 이와 같은 방식으로 구현됩니다.

## 학습

학습하는 코드는 기존 실습들과 동일하기 때문에 마지막 결과만 살펴보도록 하겠습니다.

In [14]:
from torch.optim import Adam

device = torch.device("mps")

lr = 0.001
model = model.to('mps')
loss_fn = nn.BCEWithLogitsLoss()

optimizer = Adam(model.parameters(), lr=lr)

In [15]:
import numpy as np
import matplotlib.pyplot as plt

def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('mps'), labels.to('mps')

    preds = model(inputs)
    # preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt

In [16]:
n_epochs = 50

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('mps'), labels.to('mps').float()

    preds = model(inputs)[..., 0]
    loss = loss_fn(preds, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

Epoch   0 | Train Loss: 234.17641845345497
=========> Train acc: 0.776 | Test acc: 0.749
Epoch   1 | Train Loss: 177.1879697740078
=========> Train acc: 0.826 | Test acc: 0.785
Epoch   2 | Train Loss: 150.20461484789848
=========> Train acc: 0.859 | Test acc: 0.792
Epoch   3 | Train Loss: 129.63557395339012
=========> Train acc: 0.883 | Test acc: 0.801
Epoch   4 | Train Loss: 111.76054534316063
=========> Train acc: 0.879 | Test acc: 0.777
Epoch   5 | Train Loss: 94.4647870361805
=========> Train acc: 0.936 | Test acc: 0.809
Epoch   6 | Train Loss: 77.94469979777932
=========> Train acc: 0.933 | Test acc: 0.802
Epoch   7 | Train Loss: 64.0454399511218
=========> Train acc: 0.946 | Test acc: 0.799
Epoch   8 | Train Loss: 54.0567739456892
=========> Train acc: 0.973 | Test acc: 0.806
Epoch   9 | Train Loss: 42.58272715099156
=========> Train acc: 0.972 | Test acc: 0.799
Epoch  10 | Train Loss: 34.06358318962157
=========> Train acc: 0.963 | Test acc: 0.782
Epoch  11 | Train Loss: 30.2571

학습이 안정적으로 진행되며 RNN보다 빨리 수렴하는 것을 확인할 수 있습니다.
하지만 test 정확도가 RNN보다 낮은 것을 보았을 때, overfitting에 취약하다는 것을 알 수 있습니다.